In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

import string
import math
import re
from collections import Counter

In [2]:
data = pd.read_json('gg2013.json')

In [3]:
data_df = pd.DataFrame(data)

In [4]:
# return cleaned Tweet as string
# remove stopwords, user handles, punctuation, urls

def cleanTweets(tweet):
    tt = TweetTokenizer(strip_handles=True, reduce_len=True, preserve_case=True)

    punctuation = list(string.punctuation)
    
    # strip stopwords, punctuation, url components 
    stop = stopwords.words('english') + punctuation + ['t.co', 'http', 'https', '...', '..', ':\\', 'RT', '#']

    strip_nums = re.sub("\d+", "", tweet)
    tokenized = tt.tokenize(strip_nums)
    terms_stop = [term for term in tokenized if term not in stop]
    cleaned = [term for term in terms_stop]
    cleaned = ' '.join(cleaned)
    
    return cleaned

In [5]:
data['text'] = data['text'].map(cleanTweets)

In [6]:
def findPresenters(data):
    include_tweets = ['present', 'presents', 'presenter', 'presenters', 'presenting', 'presented']
    need_terms = ['animated']
    remove_tweets = []
    remove_terms = ['#GoldenGlobes', 'Golden Globes', 'golden globes', '#GoldenGlobe', 'Golden Globe', '#goldenglobes']
    
    presenter = [];

    for tweet in data['text']:
        if any(term in tweet for term in include_tweets) and all(term not in tweet for term in remove_tweets) and all(term in tweet for term in need_terms):
            for term in remove_terms:
                tweet = re.sub(term, "", tweet)
                presenter.append(tweet)            
            
    bgrams = [];

    for tweet in presenter:
        tweet = re.findall('([A-Z][a-z]+)', tweet)
        if tweet:
            bgrams += list(nltk.bigrams(tweet))
        
    fdist = nltk.FreqDist(bgrams)
    
    return fdist.most_common(20)

In [7]:
print(findPresenters(data))

[(('Sacha', 'Baron'), 12), (('Baron', 'Cohen'), 12), (('George', 'Bush'), 6), (('What', 'Great'), 6), (('Cohen', 'Tarantino'), 6), (('Cohen', 'Brave'), 6)]
